In [2]:
import pmdarima as pm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv('No_abnormality_time/time2.csv', index_col=3)
length = data.qty.shape[0]
scaler = MinMaxScaler()
data.qty = scaler.fit_transform(data.qty.values.reshape(-1, 1))
train_data = data.qty[:int(length * 0.8)]
test_data = data.qty[int(length * 0.8):]

In [3]:
md = pm.auto_arima(train_data, start_p=1, start_q=1,
                      information_criterion='aic',
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=30,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # No Seasonality
                      start_P=0, 
                      D=0)

In [4]:
import pylab as plt
import numpy as np
def wmapes(y_true, y_pred):
    return 1 - np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)


In [5]:
test_data = scaler.inverse_transform(test_data.values.reshape(-1, 1))
test_data_predict = scaler.inverse_transform(md.predict(n_periods=test_data.shape[0]).values.reshape(-1, 1))

In [6]:
wmapes(test_data, test_data_predict)
import pickle
pickle.dump(md, open('No_abnormality_time/model.pkl', 'wb'))

0.7396712170580058